In [1]:
import os 
import json 
import re
import numpy as np

from collections import Counter, OrderedDict

np.set_printoptions(precision=2)

# Extract utilities

## Function

In [2]:
bound = 0.005
# Go from bid -> utility
def get_utility(bid, pref):
    util = 0
    issuelist = list(pref.keys())
    for issueindex, issue in enumerate(issuelist):
        # values of issue in given bid
        value = pref[issue][bid[issueindex]]
        # weight of issue in given bid
        weight = pref[issue]['weight']
        util += weight*value
    return util

# Utility change to discrete move type
def delta_mapping(delta_util):
    if (abs(delta_util[0]) <= bound and abs(delta_util[1]) <= bound):
        return 'silent'
    if (abs(delta_util[0]) <= bound and delta_util[1]) > bound:
        return 'nice'
    if (delta_util[0] <= 0 and delta_util[1] <= 0):
        return 'unfortunate'
    if (delta_util[0] > 0 and delta_util[1] <= 0):
        return 'selfish'
    if (delta_util[0] > 0 and delta_util[1] > 0):
        return 'fortunate'
    if (delta_util[0] <= 0 and delta_util[1] > 0):
        return 'concession'
    print(delta_util)

# List of bids to list of discrete moves
def discritized_mapping(agent_bids):
    mapped_utils_discrete = []
    prev_utils = agent_bids[0]
    for new_utils in agent_bids[1:]:
        delta_util1 = new_utils[0] - prev_utils[0]
        delta_util2 = new_utils[1] - prev_utils[1]
        delta_util = (delta_util1, delta_util2)
        mapped_utils_discrete.append(delta_mapping(delta_util))
        prev_utils = new_utils
    return mapped_utils_discrete
        
    
def retrieve_all_agents_bids(train):
    # Useful structures
    all_issues = train['issues']
    pref1 = train['Utility1']
    pref2 = train['Utility2']
    all_bids = train['bids']

    mapped_utils_a1 = []
    mapped_utils_a2 = []

    # Parse utility values of bids
    for bid in all_bids:
        r = bid['round']
#         print(bid)
        # stop if the negotiation session has ended
        if 'agent1' in bid:
            bid_agent1 = bid['agent1'].split(',')
            u1_b1 = get_utility(bid_agent1, pref1)
            u2_b1 = get_utility(bid_agent1, pref2)
            # Save the bid -> utility mapping
            mapped_utils_a1.append([u1_b1, u2_b1, int(r)])
        if 'agent2' in bid:
            bid_agent2 = bid['agent2'].split(',')
            u1_b2 = get_utility(bid_agent2, pref1)
            u2_b2 = get_utility(bid_agent2, pref2)
            mapped_utils_a2.append([u2_b2, u1_b2, int(r)])


    agent1_bids = [mapped_utils_a1[i][0:2] for i in range(len(mapped_utils_a1))]
    agent2_bids = [mapped_utils_a2[i][0:2] for i in range(len(mapped_utils_a2))]

    # Discritize bidspace
    agent1_bids_discrete = discritized_mapping(agent1_bids)
    agent2_bids_discrete = discritized_mapping(agent2_bids)
    
    return (agent1_bids_discrete, agent2_bids_discrete)

# Model parameters 

## Sensor model 1: Simple agent move as possible evidences.

In [14]:
path='train/'

train_files = os.listdir(path)
agent_count_mapping = {}
for t in train_files:
    train = json.load(open(os.path.join(path, t)))
    a1_name, a2_name = re.split(r'[^A-Za-z]+', t.strip('.json'))[0:2]
    a1_bids, a2_bids = retrieve_all_agents_bids(train)
    if a1_name not in agent_count_mapping:
        agent_count_mapping[a1_name] = []
    if a2_name not in agent_count_mapping:
        agent_count_mapping[a2_name] = []
    agent_count_mapping[a1_name] = agent_count_mapping[a1_name] + a1_bids
    agent_count_mapping[a2_name] = agent_count_mapping[a2_name] + a2_bids

# Start constructing the model.
sensor_model = {}
for k, v in agent_count_mapping.items():
    cnt_bids = Counter(v)
    total = len(v)
    for key in cnt_bids:
        cnt_bids[key] /= total
    sensor_model[k] = dict(cnt_bids)
    
possible_moves = ['silent', 'concession', 'unfortunate', 'nice', 'fortunate', 'selfish']

for k, moves in sensor_model.items():
    for pm in possible_moves:
        if pm not in moves:
            moves[pm] = 0.0
    sensor_model[k] = np.array(list(dict(OrderedDict(sorted(moves.items()))).values()))

evidence_index = {k: v for v, k in enumerate(sorted(possible_moves))}
state_index = {k: v for v, k in enumerate(list(sensor_model.keys()))}
sensor_model_simple = np.array(list(sensor_model.values()))

# Verification prints.
print(state_index)
print(evidence_index)
sensor_model_simple

{'hardheaded': 0, 'random': 1, 'conceder': 2, 'tft': 3}
{'concession': 0, 'fortunate': 1, 'nice': 2, 'selfish': 3, 'silent': 4, 'unfortunate': 5}


array([[0.04, 0.  , 0.02, 0.  , 0.9 , 0.04],
       [0.42, 0.11, 0.01, 0.34, 0.  , 0.12],
       [0.08, 0.  , 0.22, 0.  , 0.4 , 0.29],
       [0.1 , 0.01, 0.08, 0.03, 0.62, 0.16]])

## Sensor model 2: Combination of agent move and opponent move as evidence

In [15]:
path='train/'

train_files = os.listdir(path)
agent_count_mapping = {}
for t in train_files:
    train = json.load(open(os.path.join(path, t)))
    a1_name, a2_name = re.split(r'[^A-Za-z]+', t.strip('.json'))[0:2]
    a1_bids, a2_bids = retrieve_all_agents_bids(train)
    
    if a1_name not in agent_count_mapping:
        agent_count_mapping[a1_name] = []
    if a2_name not in agent_count_mapping:
        agent_count_mapping[a2_name] = []
        
    a1_com = [a1_bids[i] + a2_bids[i] for i in range(min(len(a1_bids), len(a2_bids)))]
    a2_com = [a2_bids[i] + a1_bids[i] for i in range(min(len(a1_bids), len(a2_bids)))]
    
    agent_count_mapping[a1_name] = agent_count_mapping[a1_name] + a1_com 
    agent_count_mapping[a2_name] = agent_count_mapping[a2_name] + a2_com 
agent_count_mapping

# Start constructing the model.
sensor_model = {}
for k, v in agent_count_mapping.items():
    cnt_bids = Counter(v)
    total = len(v)
    for key in cnt_bids:
        cnt_bids[key] /= total
    sensor_model[k] = dict(cnt_bids)
    
pm = ['silent', 'concession', 'unfortunate', 'nice', 'fortunate', 'selfish']
possible_moves = []

for m1 in pm:
    for m2 in pm:
        possible_moves.append(m1 + m2)

for k, moves in sensor_model.items():
    for pm in possible_moves:
        if pm not in moves:
            moves[pm] = 0.0
    sensor_model[k] = np.array(list(dict(OrderedDict(sorted(moves.items()))).values()))

evidence_index = {k: v for v, k in enumerate(sorted(possible_moves))}
state_index = {k: v for v, k in enumerate(list(sensor_model.keys()))}

sensor_model= np.array(list(sensor_model.values()))
print(state_index)
print(evidence_index)
sensor_model

{'hardheaded': 0, 'random': 1, 'conceder': 2, 'tft': 3}
{'concessionconcession': 0, 'concessionfortunate': 1, 'concessionnice': 2, 'concessionselfish': 3, 'concessionsilent': 4, 'concessionunfortunate': 5, 'fortunateconcession': 6, 'fortunatefortunate': 7, 'fortunatenice': 8, 'fortunateselfish': 9, 'fortunatesilent': 10, 'fortunateunfortunate': 11, 'niceconcession': 12, 'nicefortunate': 13, 'nicenice': 14, 'niceselfish': 15, 'nicesilent': 16, 'niceunfortunate': 17, 'selfishconcession': 18, 'selfishfortunate': 19, 'selfishnice': 20, 'selfishselfish': 21, 'selfishsilent': 22, 'selfishunfortunate': 23, 'silentconcession': 24, 'silentfortunate': 25, 'silentnice': 26, 'silentselfish': 27, 'silentsilent': 28, 'silentunfortunate': 29, 'unfortunateconcession': 30, 'unfortunatefortunate': 31, 'unfortunatenice': 32, 'unfortunateselfish': 33, 'unfortunatesilent': 34, 'unfortunateunfortunate': 35}


array([[0.01, 0.  , 0.  , 0.  , 0.  , 0.02, 0.  , 0.  , 0.  , 0.  , 0.  ,
        0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.01, 0.  , 0.  , 0.  , 0.  ,
        0.  , 0.  , 0.09, 0.01, 0.07, 0.05, 0.57, 0.11, 0.02, 0.  , 0.01,
        0.  , 0.  , 0.02],
       [0.15, 0.02, 0.02, 0.05, 0.12, 0.06, 0.02, 0.  , 0.  , 0.03, 0.03,
        0.02, 0.01, 0.  , 0.  , 0.  , 0.  , 0.  , 0.06, 0.04, 0.02, 0.07,
        0.13, 0.03, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.03, 0.01, 0.  ,
        0.02, 0.03, 0.04],
       [0.01, 0.  , 0.  , 0.01, 0.04, 0.01, 0.  , 0.  , 0.  , 0.  , 0.  ,
        0.  , 0.02, 0.  , 0.05, 0.01, 0.1 , 0.05, 0.  , 0.  , 0.  , 0.  ,
        0.  , 0.  , 0.03, 0.  , 0.05, 0.01, 0.24, 0.07, 0.03, 0.01, 0.04,
        0.01, 0.14, 0.07],
       [0.05, 0.  , 0.01, 0.  , 0.  , 0.04, 0.  , 0.  , 0.  , 0.01, 0.  ,
        0.  , 0.  , 0.  , 0.04, 0.  , 0.01, 0.02, 0.  , 0.01, 0.  , 0.02,
        0.  , 0.  , 0.01, 0.  , 0.02, 0.01, 0.56, 0.03, 0.03, 0.  , 0.05,
        0.  , 0.01, 0.07]])

## State transitions

In [4]:
state_transition = np.identity(len(state_index), dtype=float)
state_transition

array([[1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.]])

# Filtering step

In [5]:
def predict_agent(bids):
    Bt = np.ones(4)*0.25
    for t, bt in enumerate(bids):
        sm1 = sensor_model[:, evidence_index[bt]]
        stm1 = np.dot(state_transition, Bt)
        Bt = np.multiply(sm1, stm1)
        Bt = Bt/sum(Bt)
    return Bt

In [13]:
path='train/'

train_files = os.listdir(path)
agent_count_mapping = {}
labels_keys = list(state_index.keys())

labels = []
preds = []
for t in train_files:
    train = json.load(open(os.path.join(path, t)))
    a1_name, a2_name = re.split(r'[^A-Za-z]+', t.strip('.json'))[0:2]
    a1_bids, a2_bids = retrieve_all_agents_bids(train)
    labels.append(a1_name)
    labels.append(a2_name)
    
    a1_com = [a1_bids[i] + a2_bids[i] for i in range(min(len(a1_bids), len(a2_bids)))]
    a2_com = [a2_bids[i] + a1_bids[i] for i in range(min(len(a1_bids), len(a2_bids)))]
    
    p1 = predict_agent(a1_com)
    p2 = predict_agent(a2_com)
    
    preds.append(labels_keys[np.argmax(p1)])
    preds.append(labels_keys[np.argmax(p2)])
    
    if preds[-1] != labels[-1]: 
        print("prediction: {}, labels: {}, file: {}".format(preds[-1], labels[-1], t))
        
    if preds[-2] != labels[-2]: 
        print("prediction: {}, labels: {}, file: {}".format(preds[-2], labels[-2], t))

prediction: tft, labels: hardheaded, file: hardheaded_tft1.json
prediction: tft, labels: hardheaded, file: hardheaded_tft2.json


In [7]:
c = [labels[x] == preds[x] for x in range(len(labels))]
# for i i
sum(c)/float(len(labels))

0.9411764705882353